In [2]:
import requests
from bs4 import BeautifulSoup
import json
import re
import os
import pandas as pd

In [38]:
url = 'https://understat.com/league/Bundesliga'
variable_name = 'teamsData'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
script = soup.find_all('script')
 # Nach dem Script mit 'datesData' suchen
for script in scripts:
    if  variable_name in script.string:
        json_data_raw = script.string
        break
x= json_data_raw.find('(')
y= json_data_raw.find(')')
json_data_raw=json_data_raw[x+2:y-1]
json_clean = json_data_raw.encode().decode('unicode_escape')
##in python-Dictonary umwandeln
data = json.loads(json_clean)
## in Datei abspeichern
notebook_path = r"C:\Users\Schönherr\xGModell"
filename = os.path.join(notebook_path, "alle_websiten.json")
with open(filename, "w") as outfile:
    json.dump(data, outfile, indent=4)
with open(filename, 'r') as infile:
    json_data_dates = json.load(infile)
teams_list = list(json_data_dates.values())
df = pd.DataFrame(teams_list)
team_namen = df['title'].tolist()
neu = []
for x in team_namen:
    if x.find(' ') != -1: 
        index = x.find(' ')
        neues_wort = x[:index]+ '_'+x[index+1:]
        neu.append(neues_wort)
        continue
    neu.append(x)
##Websiten-URLs kontruieren
team_url_liste = []
for team_name in neu: 
    team_url_liste.append(f"https://understat.com/team/{team_name}/2024")

In [45]:
#Für jeden Verein jede Spielwebsite webscrapen
websiten_liste= []
for website in team_url_liste:
        url=website
        variable_name1= 'datesData'
        response1= requests.get(url)
        soup1= BeautifulSoup(response1.content, "html.parser")
        scripts1 = soup1.find_all('script')
        # Nach dem Script mit 'datesData' suchen
        for script in scripts1:
            if 'datesData' in script.string:
                json_data_raw = script.string
                break
        x= json_data_raw.find('(')
        y= json_data_raw.find(')')
        json_data_raw=json_data_raw[x+2:y-1]
        json_clean = json_data_raw.encode().decode('unicode_escape')
        ##in python-Dictonary umwandeln
        data = json.loads(json_clean)
        ## in Datei abspeichern
        notebook_path = r"C:\Users\Schönherr\xGModell"
        filename = os.path.join(notebook_path, "websiten.json")
        with open(filename, "w") as outfile:
            json.dump(data, outfile, indent=4)
        with open(filename, 'r') as infile:
            json_data_dates = json.load(infile)
        ##alle Websitenurl heruausfinden
        df = pd.DataFrame(json_data_dates)
        website_url= df.iloc[:,0].to_list()
        url ='https://understat.com/match/'
        for x in range(0,34):
            websiten_liste.append(url + website_url[x])
    ## Die Liste aller Spielstatistikenwebsiten für die Spiele des FC Augsburg in der Saison 24/25
websiten_liste = list(set(websiten_liste))

In [ ]:
#alle Schüsse in der Saison 24/25 zu der Json-Datei hinzufügen
notebook_path = r"C:\Users\Schönherr\xGModell"
if not os.path.exists(notebook_path):
    os.makedirs(notebook_path)
filename = os.path.join(notebook_path, "shots.json")
alle_shots=[]
for x in websiten_liste:
    url  = x
    variable_name='shotsData'
    response= requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    scripts = soup.find_all('script')
    for script in scripts:
        if variable_name in script.text:
            ##Json-Rohdaten finden
            start = script.text.find('JSON.parse("') + len('JSON.parse("')
            end = script.text.find('");', start)
            ##Rohdaten so manipulieren, dass nur noch die escape-Daten vorhanden sind
            json_raw = script.text[start:end]
            x= json_raw.find('(')
            y= json_raw.find(')')
            json_raw=json_raw[x+2:y-1]
            ## von escape Rohdaten in normaler JSON-Form umwandlen
            json_clean = json_raw.encode().decode('unicode_escape')
            ##in python-Dictonary umwandeln
            data = json.loads(json_clean) 
            alle_shots.append(data)
            break
notebook_path = r"C:\Users\Schönherr\xGModell"
with open(filename, "w", encoding='utf-8') as outfile:
    json.dump(alle_shots, outfile, indent=4, ensure_ascii=False)
    # Zum Testen Datei laden und ausgeben
with open(filename, 'r', encoding='utf-8') as infile:
    json_data_shots = json.load(infile)
print(json_data_shots)